# Sediment resuspension parameterization

Based on tidal speeds from Jacqui-Lee

In [1]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import scipy.io

##### Parameters

In [2]:
# domain dimensions
imin, imax = 1479, 2179
jmin, jmax = 159, 799

##### Load files:

In [3]:
mesh         = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_lon     = np.array(mesh.variables['nav_lon'])
mesh_lat     = np.array(mesh.variables['nav_lat'])
mesh_bathy   = np.array(mesh.variables['hdept'][0])
tmask        = np.array(mesh.variables['tmask'])[0,:,:,:]
bathy_masked = np.ma.masked_where((mesh_bathy > 0.1), mesh_bathy)

In [4]:
# Tidal speed matrix provided by Jacqui-Lee Epstein and Stephanie Waterman (extracted from Carrere and Lyard, 2003)
mat         = scipy.io.loadmat('/ocean/brogalla/GEOTRACES/data/mean_tide_speed.mat')
tidal_speed = mat['mean_speed']

ts_mesh     = nc.Dataset('/ocean/brogalla/GEOTRACES/data/hsquare.nc')
ts_lons     = np.array(ts_mesh.variables['nav_lon'])
ts_lats     = np.array(ts_mesh.variables['nav_lat'])

##### Functions

In [5]:
def interp_np(nav_lon, nav_lat, var_in, lon_ANHA12, lat_ANHA12):
    # Interpolate some grid to ANHA12 grid
    
    from scipy.interpolate import griddata

    LatLonPair = (nav_lon.flatten(), nav_lat.flatten())
    var_out = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='linear')
    # Take nearest neighbour interpolation to fill nans
    var_fill = griddata(LatLonPair, var_in.flatten(), (lon_ANHA12, lat_ANHA12), method='nearest')
    var_out[np.isnan(var_out)] = var_fill[np.isnan(var_out)]
    
    return var_out

##### Calculations

In [6]:
tidal_speed_ANHA12 = interp_np(ts_lons, ts_lats, tidal_speed, mesh_lon, mesh_lat)
tidal_speed_ANHA12[np.isnan(tidal_speed_ANHA12)] = 0
tidal_speed_ANHA12[tidal_speed_ANHA12 < 0.01]    = 0 # mask tidal speeds slower than 1 cm/s

In [7]:
gamma            = 0.065
alpha_0          = 0.3e-6*9*0.8*0.055/gamma
erosion_rate     = alpha_0*gamma*(1-np.exp(-tidal_speed_ANHA12**2/gamma))
log_erosion_rate = np.log10(erosion_rate)

/home/brogalla/miniconda3/envs/julab/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


# Write to file:

In [ ]:
file_write = xr.Dataset(
    {'er_rate': (("y","x"), erosion_rate)}, 
    coords = {
        "y": np.zeros(2400),
        "x": np.zeros(1632),
    },
    attrs = {
        'long_name':'Erosion rate',
        'units':'kg/m2/s',
    }
)

In [ ]:
file_write.to_netcdf('/ocean/brogalla/GEOTRACES/data/erosion_rate-20211004.nc')